# 월간 데이콘 법원 판결 예측 AI 경진대회
Dataset Info.
* ID : 사건 샘플 ID
* first_party : 사건의 첫 번째 당사자
* second_party : 사건의 두 번째 당사자
* facts : 사건 내용
* first_party_winner : 첫 번째 당사자의 승소 여부 (0 : 패배, 1 : 승리)

**첫 번째 당사자의 승소 여부 예측하기**

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import nltk # 문장 토크나이저

# 영어 불용어 - 불용어 모아 놓은 리스트 다운로드해 제거
nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize # 토큰화
from nltk.stem.porter import PorterStemmer # 어근 동일화 <-> 이거 말고도 "Lancaster Stemmer"

# 표제어 추출
from nltk.stem import WordNetLemmatizer

# 정규표현 처리
import re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [4]:
# 영어 데이터 전처리 함수
stops = set(stopwords.words('english'))
stemmer = nltk.stem.SnowballStemmer('english')

def cleaning(str):
    replaceAll = str

    # 특수문자 및 기호 등 필요없는 문자 제거
    only_english = re.sub('[^a-zA-Z]', ' ', replaceAll)

    # 대소문자 모두 소문자로 통일
    no_capitals = only_english.lower().split()

    # 불용어(분석에 필요없는 토큰) 제거
    no_stops = [word for word in no_capitals if not word in stops]

    # 어근 추츨을 통한 텍스트 정규화 작업
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    return ' '.join(stemmer_words)

In [5]:
train = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/test.csv')

In [ ]:
train['facts_lst'] = 0 # 문장 토큰화
train['facts_word'] = 0 # 단어 토큰화

for i in range(2478):
  train.facts_lst[i] = nltk.sent_tokenize(train.facts[i])
  train.facts_word[i] = word_tokenize(train.facts[i])

## 파생변수 생성

* first_party_win_percent	: 첫 번째 당사자가 승소할 확률. 출현 빈도가 2이상일때만 반영함. 1일때는 0.5
* test에서는 first_party가 train의 first_party와 겹치는 경우에는 해당 사람의 승소할 확률을 반영함. 겹치지 않는 경우는 0.5로 반영함.

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [7]:
# winner: 해당 사건에서 승소한 사람
# frequency: winner의 총 등장 횟수
# win_percentage: 승소한 사람이 다른 사건에서도 승소할 확률

train['winner']=0
train['winner_frequency']=0
train['win_percentage']=0

for i in range(2477):
  if train['first_party_winner'][i]==1:
    train['winner'][i] = train['first_party'][i]
  else:
    train['winner'][i] = train['second_party'][i]

for i in range(2477):
  train['winner_frequency'][i] = len(train.loc[train['first_party'] == train['winner'][i]]) + len(train.loc[train['second_party'] == train['winner'][i]])
  train['win_percentage'][i]= len(train.loc[train['winner'] == train['winner'][i]]) /(len(train.loc[train['first_party'] == train['winner'][i]]) + len(train.loc[train['second_party'] == train['winner'][i]]))

In [8]:
# train['loser']=0
# train['lose_frequency']=0
# train['lose_percentage']=0

# for i in range(2477):
#   if train['first_party_winner'][i]==0:
#     train['loser'][i] = train['first_party'][i]
#   else:
#     train['loser'][i] = train['second_party'][i]

# for i in range(2477):
#   train['lose_frequency'][i] = len(train.loc[train['first_party'] == train['loser'][i]]) + len(train.loc[train['second_party'] == train['loser'][i]])
#   train['lose_percentage'][i]= len(train.loc[train['loser'] == train['loser'][i]]) /(len(train.loc[train['first_party'] == train['loser'][i]]) + len(train.loc[train['second_party'] == train['loser'][i]]))

In [9]:
# first_party_frequency: first_party가 나타난 빈도
# first_party_win_percent: first_party가 승소할 확률 / 출현 빈도가 2이상일때만 반영함. 1일때는 0.5
train['first_party_win_percent'] = 0
train['first_party_frequency'] = 0

for i in range(2477):
  train['first_party_frequency'][i] = len(train.loc[train['first_party'] == train['first_party'][i]]) + len(train.loc[train['second_party'] == train['first_party'][i]])

  if (len(train.loc[train['first_party'] == train['first_party'][i]]) + len(train.loc[train['second_party'] == train['first_party'][i]])) > 1:
    train['first_party_win_percent'][i]= len(train.loc[train['winner'] == train['first_party'][i]]) /(len(train.loc[train['first_party'] == train['first_party'][i]]) + len(train.loc[train['second_party'] == train['first_party'][i]]))
  else:
    train['first_party_win_percent'][i] = 0.5

In [11]:
#train=train.drop(['winner',	'winner_frequency',	'win_percentage',	'loser',	'lose_frequency',	'lose_percentage'],axis=1)
train=train.drop(['winner',	'winner_frequency',	'win_percentage'],axis=1)

In [43]:
train_first_party = train[['first_party','first_party_win_percent']]
train_first_party=train_first_party.drop_duplicates()

In [44]:
train_first_party.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2470    False
2473    False
2474    False
2475    False
2477    False
Length: 2110, dtype: bool

병합

In [45]:
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/test.csv')
test_first_party = test['first_party']
test_first_party

0                                                 Salerno
1                  Milberg Weiss Bershad Hynes and Lerach
2       No. 07-582\t Title: \t Federal Communications ...
3                                         Harold Kaufman 
4                                                  Berger
                              ...                        
1235                Haitian Centers Council, Inc., et al.
1236                                              Whitman
1237                  Linda A. Matteo and John J. Madigan
1238        Washington State Apple Advertising Commission
1239                                     Theodore Stovall
Name: first_party, Length: 1240, dtype: object

In [47]:
test_first_party_병합 = pd.merge(test_first_party, train_first_party, how='left')

In [50]:
test_first_party_병합.isnull().sum()

first_party                  0
first_party_win_percent    971
dtype: int64

In [51]:
test_first_party_병합 = test_first_party_병합.fillna(0.5)

In [52]:
test_first_party_병합.head(20)

,first_party,first_party_win_percent
0,Salerno,0.50000
1,Milberg Weiss Bershad Hynes and Lerach,0.50000
2,No. 07-582\t Title: \t Federal Communications ...,0.50000
3,Harold Kaufman,0.50000
4,Berger,0.50000
5,Air Line Pilots Association,0.50000
6,Traffic Stream (BVI) Infrastructure Ltd.,0.50000
7,NRG Power Marketing LLC et al.,0.50000
8,United States,0.57868
9,United States Catholic Conference,0.50000


In [53]:
#test에 새로운 컬럼 추가
test['first_party_win_percent'] = test_first_party_병합['first_party_win_percent']

In [54]:
test

,ID,first_party,second_party,facts,first_party_win_percent
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...,0.5
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...,0.5
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa...",0.5
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...,0.5
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a...",0.5
...,...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...,0.5
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...,0.5
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...,0.5
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu...",0.5
